# Reliability Analysis

## Awareness
1. Awareness of reliability concepts and applications.
2. Understanding reliability concepts, methods, and applications.

Reliability is the probability of an item performing its intended function **without failure** for a specified period of time. Its key concepts:
+ Probability: the likelihood of success.
+ Function: to cut, heat, or rotate.
+ Satisfactory: meet specifications with acceptable compliance.
+ Specific period: minutes, days, months, or cycle.
+ Specific condition: temperature, speed, or pressure.

Source: [What is Reliability?](https://asq.org/quality-resources/reliability)

**Failure rate**: the number of times of an item fails in a specified period of time.
$$ \lambda = \frac{Number\ of\ Failures}{Total\ Period\ of\ Time} $$

**Time to Failure (MTTF)**: the duration until an item to fail.
$$ TTF = Total\ Durations\ of\ Operation $$

**Mean Time to Failure (MTTF)**: the average time for an item to fail.
$$ MTTF = \frac{Total\ Durations\ of\ Operation}{Total\ Numbers\ of\ Time} $$

**Mean Time to Repair (MTTR)**: the average time required to repair.
$$ MTTR = \frac{Total\ Durations\ of\ Maintenance}{Total\ Numbers\ of\ Repairs} $$

Source: [MTBF and Product Reliability](https://ftp.automationdirect.com/pub/Product%20Reliability%20and%20MTBF.pdf)

## Knowledge
1. Apply reliability methods effectively.
2. Understand basic reliability analysis methods.
3. Able to use reliability software or packages for implementing reliability models.